In [6]:
import requests
import pandas as pd
import plotly.graph_objs as go
from datetime import datetime, timedelta

# Replace this with your actual API key
api_key = 'UFvGKAnwQkIgSwLrTndbScEG3Z4J4DeypSJiC5FJ'
start_date = '2024-05-01'
end_date = (datetime.strptime(start_date, '%Y-%m-%d') + timedelta(days=7)).strftime('%Y-%m-%d')

# Construct the API URL
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={api_key}'

# Fetch the data from NASA's NeoWs API
response = requests.get(url)
data = response.json()

# Check for errors in the response
if response.status_code != 200:
    raise Exception(f'Error: {data.get("error_message", "Unknown Error")}')
else:
    print(f'Data retrieved successfully for the period: {start_date} to {end_date}')

# Process the data
near_earth_objects = data['near_earth_objects']
objects_list = []
for date, neos in near_earth_objects.items():
    for neo in neos:
        close_approach = neo['close_approach_data'][0]
        neo_data = {
            'name': neo['name'],
            'id': neo['id'],
            'close_approach_date': date,
            'relative_velocity_km_s': float(close_approach['relative_velocity']['kilometers_per_second']),
            'miss_distance_km': float(close_approach['miss_distance']['kilometers']),
            'orbiting_body': close_approach['orbiting_body'],
            'is_potentially_hazardous': neo['is_potentially_hazardous_asteroid'],
            'latitude': float(close_approach['orbiting_body'] == 'Earth') * 10.0,
            'longitude': float(close_approach['orbiting_body'] == 'Earth') * 10.0
        }
        objects_list.append(neo_data)

# Create a DataFrame
df = pd.DataFrame(objects_list)

# Example latitude/longitude values for visualization
import random

random.seed(42)
df['latitude'] = [random.uniform(-90, 90) for _ in range(len(df))]
df['longitude'] = [random.uniform(-180, 180) for _ in range(len(df))]

# Globe visualization using Plotly
fig = go.Figure()

# Add NEOs as scattergeo
fig.add_trace(go.Scattergeo(
    lon=df['longitude'],
    lat=df['latitude'],
    text=df['name'],
    mode='markers',
    marker=dict(
        size=[5 if ph else 2 for ph in df['is_potentially_hazardous']],
        color=['red' if ph else 'green' for ph in df['is_potentially_hazardous']],
        opacity=0.8
    ),
    name='Near-Earth Objects'
))

# Configure globe appearance
fig.update_geos(
    projection_type='orthographic',
    showcoastlines=True,
    showland=True,
    landcolor='rgb(212, 212, 212)',
    coastlinecolor='rgb(63, 63, 63)',
    projection_rotation=dict(lon=-100, lat=45),
    lataxis_showgrid=True,
    lonaxis_showgrid=True
)

fig.update_layout(
    title_text='Near Earth Objects Orbiting Earth',
    showlegend=True,
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='orthographic'
    )
)

fig.show()


Data retrieved successfully for the period: 2024-05-01 to 2024-05-08
